# A Complete Introduction and Walkthrough - First Try

## 1. Costa Rican Household Poverty Level Prediction
이 대회의 목적은 **개인과 가정의 특성을 모두 사용해 가정의 가난 정도를 예측**하는 머신러닝 모델을 만드는 것이다.     
이 노트북은 다음과 같은 순서로 진행된다:
- 문제를 살펴보기
- Exploratory Data Analysis(EDA)
- Feature Engineering
- 여러 개의 머신러닝 모델 설계
- 모델 선택
- 모델 최적화
- 모델의 출력 살펴보기
- conclusion

### 1.1. Problem and Data Explanation
이 대회에서는 `train.csv`와 `test.csv` 두 개의 파일이 제공된다. training set은 9557개의 로우와 143개의 컬럼을 가진 반면 testing set은 23856개의 로우와 142개의 컬럼을 가지고 있다. 각각의 로우는 하나의 개인을 나타내며 각 컬럼은 개인에게 고유한 피처이거나 개인의 가구에 대한 특징이다. training set은 추가로 하나의 컬럼은 갖고 있는데, `Target`이다. 이것은 대회의 Label값으로 가난의 정도를 1-4로 표현한다. 1이 가장 가난함을 뜻한다.

이것은 지도학습-다중클래스분류-머신러닝 문제이다:
* 지도학습(Supervised) : training data가 label과 함께 제공되는 것
* 다중 분류(Multi-class classification) : Label이 4개의 클래스로 나누어진 이산적인(discrete) 값

#### 1.1.1. Objective

이 대회의 목적은 다시 말하다싶이 가정의 빈곤 정도를 예측하는 것이다. 우리는 개인에 대한 데이터를 제공받는데, 각 개인에 대한 피처는 물론 그들의 가정에 대한 정보도 포함되어 있다. 제대로된 데이터셋을 만들기 위해서는, 각 가정에 대해 개별 데이터를 집계를 해야한다. 더구나 test set에서 각 개인에 대한 예측을 해야하지만 "점수에 가장(household)만 점수에 사용된다"는 말로 보아 우리는 가정/가계를 기준으로 가난 정도를 예측해야함을 뜻한다.

**Important Note: training data에서 가정의 모든 일원들은 같은 Label값을 가져야 하지만, 동일한 가구의 개인이 다른 라벨을 가지고 있는 오류(error)가 있을 수 있다. 이 경우, `parentescol == 1.0`인 로우로 식별할 수 있는 각 가장의 라벨을 사용하라고 한다. 자세한 것은 노트북에서 확인!**

`Target` 값은 가난의 정도를 의미한다:
* 1 = 극단적 빈곤
* 2 = 적당히 빈곤
* 3 = 취약 가구
* 4 = 취약하지 않은 가구

143개의 컬럼에 대한 설명은 [대회의 다큐먼트](https://www.kaggle.com/c/costa-rican-household-poverty-prediction/data)에서 확인할 수 있다. 몇몇 피처에 대해 설명하면 다음과 같다:
* `Id` : 개인을 구분하기 위한 식별자. 모델 학습 시 사용하지는 않을 예정
* `idhogar` : 각 가정을 구분하기 위한 식별자. 이 변수는 피처는 아니지만 같은 식별자를 가진 개인끼리 묶어 하나의 가정으로 그룹핑할 때 사용할 예정.
* `parentesco1` : 해당 사람이 가정의 가장인지
* `Target` : 라벨값. 같은 가정이면 모든 구성원은 같은 값을 가진다.

우리가 모델을 만들 때, 각 가정을 나타내는 라벨, 가장의 빈곤 정도를 가지고 가정을 바탕으로 학습을 시킬 것이다. 